In [31]:
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [63]:
movie_df = pd.read_csv("naver_movie/naver_movie_2170.csv")
print(len(movie_df))
movie_df.head(2)

15190


,title,story,genre
0,검은 집,10년차 성북동 캣맘 ‘냥이랑나랑’은 길음동 재개발 지역에 들어간다.[제19회 서울...,다큐멘터리
1,검은 선,"탁한 물에서 낚시하는 여자, 고통받는 자연, 무엣진의 부서진 기도문, 모든 것이 얇...",다큐멘터리


In [33]:
X_train, X_test, y_train, y_test = train_test_split(movie_df.story, movie_df.genre, test_size=0.1, random_state=1)
len(X_train), len(X_test), len(y_train), len(y_test)

(13671, 1519, 13671, 1519)

In [61]:
movie_df.groupby("genre").size()

genre
공포       2170
다큐멘터리    2170
드라마      2170
애니메이션    2170
액션       2170
에로       2170
코미디      2170
dtype: int64

In [90]:
naver_movie_df.to_csv("naver_movie/naver_movie_code.csv", index=False)

In [91]:
movie_df = pd.read_csv("naver_movie/naver_movie_code.csv")
print(len(movie_df))
movie_df.head(2)

15190


,title,story,code
0,더 퍼지: 최초의 시작,한 부부가 오랜 고향 로스앤젤레스를 떠나 아픈 기억을 뒤로 한 채 새로운 삶을 살기...,1
1,더 퍼지:거리의 반란,"1년 전, 단 하나뿐인 아들을 잃은 뒤 복수를 위해 나선 한 남자갑작스런 차 고장으...",1


In [93]:
X_train, X_test, y_train, y_test = train_test_split(movie_df.story, movie_df.code, test_size=0.1, random_state=1)
len(X_train), len(X_test), len(y_train), len(y_test)

(13671, 1519, 13671, 1519)

In [94]:
clf = Pipeline([
    ('vect', TfidfVectorizer()), 
    ('clf', MultinomialNB(alpha=0.01)),
])

In [95]:
%%time
model = clf.fit(X_train.values.astype("str"), y_train)

Wall time: 5.6 s


In [96]:
y_pred = model.predict(X_test)

In [97]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.63      0.78      0.70       220
           2       0.79      0.80      0.80       220
           3       0.49      0.46      0.48       228
           4       0.74      0.58      0.65       192
           5       0.68      0.73      0.70       229
           6       0.91      0.91      0.91       201
           7       0.56      0.52      0.54       229

    accuracy                           0.68      1519
   macro avg       0.69      0.68      0.68      1519
weighted avg       0.68      0.68      0.68      1519



In [101]:
classification_dict = {
    1:"공포",
    2:"다큐멘터리",
    3:"드라마",
    4:"애니메이션",
    5:"액션",
    6:"에로",
    7:"코미디"
}

In [112]:
stories = [
    "한 여름 한적한 시골길에서 벌어진 살인, 사건은 점점 미궁속으로 빠져드는데",
    "웃겨죽겠네 아주. 좀 잘 좀 맞춰봐라. 재밌는거 없나?",
    "소년은 새로운 세계로 모험을 나선다",
]

datas = {
    "story": stories,
    "code": model.predict(stories)
}

df = pd.DataFrame(datas)
df["genre"] = df["code"].apply(lambda data: classification_dict[data])
df["proba"] = df["story"].apply(lambda data: round(max(model.predict_proba([data])[0]), 2))
df

,story,code,genre,proba
0,"한 여름 한적한 시골길에서 벌어진 살인, 사건은 점점 미궁속으로 빠져드는데",3,드라마,0.46
1,웃겨죽겠네 아주. 좀 잘 좀 맞춰봐라. 재밌는거 없나?,7,코미디,0.44
2,소년은 새로운 세계로 모험을 나선다,4,애니메이션,0.79


In [114]:
with open("naver_movie/genre.pkl", "wb") as f:
    pickle.dump(model, f)

In [116]:
with open("naver_movie/genre.pkl", "rb") as f:
    load_model = pickle.load(f)

In [99]:
model.predict_proba(["한 여름 한적한 시골길에서 벌어진 살인, 사건은 점점 미궁속으로 빠져드는데"])

array([[0.20567528, 0.01361689, 0.45539929, 0.04875674, 0.20085368,
        0.05967444, 0.01602367]])

In [100]:
max(model.predict_proba(["한 여름 한적한 시골길에서 벌어진 살인, 사건은 점점 미궁속으로 빠져드는데"]))

array([0.20567528, 0.01361689, 0.45539929, 0.04875674, 0.20085368,
       0.05967444, 0.01602367])

In [41]:
max(model.predict_proba(["한 여름 한적한 시골길에서 벌어진 살인, 사건은 점점 미궁속으로 빠져드는데"])[0])

0.43614777053318743

In [42]:
import pickle

In [43]:
with open("naver_movie/genre.pkl", "wb") as f:
    pickle.dump(model, f)

In [49]:
with open("naver_movie/genre.pkl", "rb") as f:
    load_model = pickle.load(f)

In [117]:
load_model.predict_proba(["한 여름 한적한 시골길에서 벌어진 살인, 사건은 점점 미궁속으로 빠져드는데"])[0]

array([0.20567528, 0.01361689, 0.45539929, 0.04875674, 0.20085368,
       0.05967444, 0.01602367])

In [118]:
load_model.predict(["한 여름 한적한 시골길에서 벌어진 살인, 사건은 점점 미궁속으로 빠져드는데"])[0]

3

In [119]:
result = {}

In [122]:
result["genre"] = classification_dict[load_model.predict(["한 여름 한적한 시골길에서 벌어진 살인, 사건은 점점 미궁속으로 빠져드는데"])[0]]

In [123]:
result

{'genre': '드라마'}